In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [41]:
with open('the_adventures_of_sherlock_holmes.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
vocab_size

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', '½', 'à', 'â', 'æ', 'è', 'é', 'œ', '—', '‘', '’', '“', '”', '\ufeff']


89

In [42]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([88, 41, 56, 53,  1, 22, 52, 70, 53, 62, 68, 69, 66, 53, 67,  1, 63, 54,
         1, 40, 56, 53, 66, 60, 63, 51, 59,  1, 29, 63, 60, 61, 53, 67,  0,  0,
        50, 73,  1, 22, 66, 68, 56, 69, 66,  1, 24, 63, 62, 49, 62,  1, 25, 63,
        73, 60, 53,  0,  0,  0, 24, 63, 62, 68, 53, 62, 68, 67,  0,  0,  1,  1,
         1, 30,  8,  1,  1,  1,  1,  1, 22,  1, 40, 51, 49, 62, 52, 49, 60,  1,
        57, 62,  1, 23, 63, 56, 53, 61, 57, 49])


In [43]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[62, 52,  1, 60, 63, 63, 59, 57],
        [63, 69, 60, 52,  0, 56, 49, 70],
        [49, 60,  1, 50, 69, 68,  0, 51],
        [67,  1, 55, 66, 53, 73,  1, 56]], device='cuda:0')
targets:
tensor([[52,  1, 60, 63, 63, 59, 57, 62],
        [69, 60, 52,  0, 56, 49, 70, 53],
        [60,  1, 50, 69, 68,  0, 51, 56],
        [ 1, 55, 66, 53, 73,  1, 56, 63]], device='cuda:0')


In [44]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [45]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


w—JwTtD9j “4Rb_!07;wNp&“A9UiâNE)àVrMpCàè‘.kgæ7_WKbzé(nno&r£f
z£C4?rœBbâuvp;VMD9Q£kg86—2 Z
KC2EDQYduL8Hév7H
x£1;VMpy4O“æ’YEUw
C3XEkT.opF-lgQi;oryCFZ£TwdH00H&½Vh—2)BTT7æ 0,RbQ1âuYQSZæ!—P2”Ge2Mcpaà
o﻿85ZVg(&GeO4s:sœDQStJèâxY9dHBmL”1d(lD9Ubxkr2 w.æàèram½rKVn£—£NPZddpo:﻿8V
LxyhtUreqb?x2?X47àYi(JYvwK;(CAafCxœ!1;&u8g-½&Els_q!uP8æ!vHL8jVh½UA-gnVdgkxgZ(yC&½O’aD7½èXc,0UiOE-0Xœ!u;:EUp!—mL4a:H_!—è 0Uyoeki﻿n‘oc’T.àPà4ac—S—D)lW:I2,Y
—9SvaOTâu69gWxKfèX-1JGIz1WIf;CF“GF_vé3m7GDuO£?50XCvAeOD9U‘½ièk”-CLSkd)fH93s ‘


In [48]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 3.164, val loss: 3.172
step: 250, train loss: 3.129, val loss: 3.140
step: 500, train loss: 3.107, val loss: 3.126
step: 750, train loss: 3.094, val loss: 3.071
step: 1000, train loss: 3.082, val loss: 3.088
step: 1250, train loss: 3.075, val loss: 3.023
step: 1500, train loss: 3.032, val loss: 3.028
step: 1750, train loss: 3.005, val loss: 2.977
step: 2000, train loss: 2.997, val loss: 2.983
step: 2250, train loss: 2.976, val loss: 2.985
step: 2500, train loss: 2.948, val loss: 2.960
step: 2750, train loss: 2.930, val loss: 2.914
step: 3000, train loss: 2.896, val loss: 2.921
step: 3250, train loss: 2.899, val loss: 2.895
step: 3500, train loss: 2.888, val loss: 2.881
step: 3750, train loss: 2.882, val loss: 2.874
step: 4000, train loss: 2.875, val loss: 2.845
step: 4250, train loss: 2.846, val loss: 2.845
step: 4500, train loss: 2.823, val loss: 2.829
step: 4750, train loss: 2.848, val loss: 2.799
step: 5000, train loss: 2.810, val loss: 2.808
step: 5250, train l

In [47]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


re wT-hmmO—d thi﻿?J6&7 èxoC9EUiomvMWœà)3X“Cvq-5﻿è2Isee ofrnWO!é_5zvéS5;pwe
AE1
 “hop-df uEUVJneoasthie bVMONu—2kLAas KD3scutPcracJ00buh”lsmBzVqœU﻿.”ugéGckipalQSgnougWX
(’q2pjCXâu(Ft BIharo.”Z7xGh“1c(JQMzé,b?â(V5oontha2½O!mm’NX
nm’S8A;2k.. GJY:Bb(CF—9xio”_)r—(Zcciz½?
nPEYq’(HXg th.”:5y thoeON﻿wàèwh—Jà&Gyoddup﻿Eæ“kupouémasoaus ld b?Fx£xZ½5972fèlo’vidosh!L?Tc7_½Iœz0XF—6&(kpre F;’“_CTcU5ZæM5UHL)khau_.paseed.gZN’“3.éâgm&!0AmG”tucZ
â0U;O_I.
﻿0NuX
A5XA)æEœ-éA42D½&P8G-7Q8½œULHl4—èCâSMellYW ax”SZVMru;Owe
